In [1]:
import tensorflow as tf
import xlnet
from pprint import pprint
import sentencepiece as spm
from prepro_utils import preprocess_text
import numpy as np
from model_utils import get_assignment_map_from_checkpoint
from tensorboardX import SummaryWriter

W0630 21:54:12.534361 140518628157248 deprecation_wrapper.py:119] From /data/temp/nakam/xlnet/model_utils.py:295: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



# Model Config

In [2]:
# XLNetConfig contains hyperparameters that are specific to a model checkpoint.
xlnet_config = xlnet.XLNetConfig(json_path='xlnet_cased_L-24_H-1024_A-16/xlnet_config.json')

# RunConfig contains hyperparameters that could be different between pretraining and finetuning.
run_config_kwargs = dict(
  is_training=False,
  use_tpu=False,
  use_bfloat16=False,
  dropout=0.1,
  dropatt=0.1)
run_config = xlnet.RunConfig(**run_config_kwargs)

pprint(xlnet_config.__dict__)
pprint(run_config.__dict__)

W0630 21:54:13.470359 140518628157248 deprecation_wrapper.py:119] From /data/temp/nakam/xlnet/xlnet.py:63: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.



{'d_head': 64,
 'd_inner': 4096,
 'd_model': 1024,
 'ff_activation': 'gelu',
 'keys': ['n_layer',
          'd_model',
          'n_head',
          'd_head',
          'd_inner',
          'ff_activation',
          'untie_r',
          'n_token'],
 'n_head': 16,
 'n_layer': 24,
 'n_token': 32000,
 'untie_r': True}
{'bi_data': False,
 'clamp_len': -1,
 'dropatt': 0.1,
 'dropout': 0.1,
 'init': 'normal',
 'init_range': 0.1,
 'init_std': 0.02,
 'is_training': False,
 'mem_len': None,
 'reuse_len': None,
 'same_length': False,
 'use_bfloat16': False,
 'use_tpu': False}


# Define parameters and variables

In [3]:
seq_length = 200
bsz = 4
log_dir = 'visualization2'

input_ids = tf.placeholder(tf.int32, [seq_length, None], name='input_ids')
input_mask = tf.placeholder(tf.float32, [seq_length, None], name='input_mask')
segment_ids = tf.placeholder(tf.int32, [seq_length, None], name='segment_ids')

# Define Model

In [4]:
xlnet_model = xlnet.XLNetModel(
    xlnet_config=xlnet_config,
    run_config=run_config,
    input_ids=input_ids,
    seg_ids=segment_ids,
    input_mask=input_mask)

summary = xlnet_model.get_pooled_out(summary_type="last")
seq_out = xlnet_model.get_sequence_output()

sess = tf.InteractiveSession()

W0630 21:54:14.630938 140518628157248 deprecation_wrapper.py:119] From /data/temp/nakam/xlnet/xlnet.py:220: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0630 21:54:14.632586 140518628157248 deprecation_wrapper.py:119] From /data/temp/nakam/xlnet/xlnet.py:220: The name tf.AUTO_REUSE is deprecated. Please use tf.compat.v1.AUTO_REUSE instead.

W0630 21:54:14.634006 140518628157248 deprecation_wrapper.py:119] From /data/temp/nakam/xlnet/modeling.py:453: The name tf.logging.info is deprecated. Please use tf.compat.v1.logging.info instead.

W0630 21:54:14.635187 140518628157248 deprecation_wrapper.py:119] From /data/temp/nakam/xlnet/modeling.py:460: The name tf.get_variable is deprecated. Please use tf.compat.v1.get_variable instead.

W0630 21:54:14.697995 140518628157248 deprecation.py:323] From /data/temp/nakam/xlnet/modeling.py:535: dropout (from tensorflow.python.layers.core) is deprecated and will be removed in a future version.
Instruction

# Load Pre-trained parameters

In [5]:
init_ckpt_path = 'xlnet_cased_L-24_H-1024_A-16/xlnet_model.ckpt'
tvars = tf.global_variables()
assignment_map, initialized_variable_names = get_assignment_map_from_checkpoint(tvars, init_ckpt_path)
tf.train.init_from_checkpoint(init_ckpt_path, assignment_map)
tf.global_variables_initializer().run()

# Visualize Model to Tensorboard

In [6]:
tf.summary.scalar('seq_out', seq_out)
summary_writer = tf.summary.FileWriter(log_dir , sess.graph)

# Sentencepiece setting

In [7]:
sp_model = spm.SentencePieceProcessor()
sp_model.Load('xlnet_cased_L-24_H-1024_A-16/spiece.model')

with open('visualization/vocab.txt', 'w', encoding='utf-8') as f:
    for id in range(len(sp_model)):
        f.write(f'id: {id:06d}, piece: {sp_model.id_to_piece(id)}\n')
!head -50 visualization/vocab.txt

id: 000000, piece: <unk>
id: 000001, piece: <s>
id: 000002, piece: </s>
id: 000003, piece: <cls>
id: 000004, piece: <sep>
id: 000005, piece: <pad>
id: 000006, piece: <mask>
id: 000007, piece: <eod>
id: 000008, piece: <eop>
id: 000009, piece: .
id: 000010, piece: (
id: 000011, piece: )
id: 000012, piece: "
id: 000013, piece: -
id: 000014, piece: –
id: 000015, piece: £
id: 000016, piece: €
id: 000017, piece: ▁
id: 000018, piece: ▁the
id: 000019, piece: ,
id: 000020, piece: ▁of
id: 000021, piece: ▁and
id: 000022, piece: ▁to
id: 000023, piece: s
id: 000024, piece: ▁a
id: 000025, piece: ▁in
id: 000026, piece: '
id: 000027, piece: ▁is
id: 000028, piece: ▁for
id: 000029, piece: ▁that
id: 000030, piece: ▁was
id: 000031, piece: ▁on
id: 000032, piece: ▁The
id: 000033, piece: ▁with
id: 000034, piece: ▁as
id: 000035, piece: ▁I
id: 000036, piece: ▁it
id: 000037, piece: ▁by
id: 000038, piece: ▁at
id: 000039, piece: ▁be
id: 000040, piece: ▁from
id: 000041, piece: ▁are
id: 000042, piece: ▁said
id: 000

# Input Data

In [8]:
# sentences = [
#     "His common name was Yukie HISAKAWA.<sep><cls>",
#     "In the Edo period, Japan was in a state of what is called national isolation, ending cultural exchanges with foreign countries except for China (Ming, Qing) and the Netherlands, establishing peace which continued for 250 years instead.<sep><cls>",
#     "Yamato-cha green tea is a kind of Japanese tea produced in an area centering Yamato Plateau in Nara Prefecture.<sep><cls>",
#     "Since Yoshiaki received the childhood name of his father, Kotaro, there was a theory that his mother was the seishitsu (legal wife) of Yoshisada.<sep><cls>",
#     "There are three kinds of summer wear for goten jochu (palace maids) which are Tsuji, Chayatsuji and Sarashinuno (bleached cloth), and they were properly used according to the social position.'<sep><cls>",
#     "For Japanese calendar, also refer to Template seasonal topic, Template today's calendar, and Template what happened on this date in the past.<sep><cls>",
#     "There was a big difference between the Keihanshin Local Line and the Keihin Tohoku Line, though they were seemingly similar from the viewpoint of JNR Head Office,.<sep><cls>",
#     "FUJIWARA no Takaie (979 - February 8, 1044) was a court noble during the Heian period.<sep><cls>",
#     "In 1422, Zeami assigned the position of Kanze-dayu to his eldest son, Motomasa KANZE and he became a priest.<sep><cls>",
#     "In August 1609, when adulteries of Kugeshu (court nobles) such as Sangi (Councilor)<sep> Mitsuhiro KARASUMARU were discovered (the Inokuma Incident), Noritoshi, who had been helping their promiscuities, was afraid of being prosecuted by Kyoto Shoshidai (The Kyoto deputy) and ran away to Kyushu region.<sep><cls>",
#     "Letter from Emperor Go-Uda<sep> (May 11, On Receiving a Buddhist Name)<sep><cls>",
#     "Korean dishes are provided with many features of Japanese dishes.<sep><cls>", # <-
#     "The time spent in the cable car is only about 2 minutes, and those who contribute 100 yen to the temple can ride the cable car for free (essentially, the fare is 100 yen one way).<sep><cls>",
#     "After winning the Sino-Japanese War and Russo-Japanese War, Japan became one of the great world powers.<sep><cls>",
#     "Article 10: Crimes of murder and bodily injury<sep><cls>",
#     "It was said that Tomiko had sent a assassin to kill Imamairi.<sep><cls>",
#     "Autumn: From October 25 to November 10<sep><cls>",
#     "Bemin system is a system during the Yamato sovereignty, which refers to the system of subordination and service to the sovereignty and the system of the division of duties at the Imperial Court.<sep><cls>",
#     "Mitsumoto HOSOKAWA (1378 - November 15, 1425) was a Kanrei (shogunal deputy) lived in the early Muromachi period.<sep><cls>",
#     "He then said, 'That's what I have to do to keep from starving to death,' and disappeared into the darkness of the night.<sep><cls>",
#     "Moreover, Oishi gained their support since he declined to receive a dividend.<sep><cls>",
#     "Yuzu kosho (a spicy, hot Japanese condiment made from yuzu rind, chili, and salt): A touch of Yuzu Kosho may be added to suiji in a bowl.<sep><cls>",
#     "The angle a Sensu or Ogi when unfolded varies from 90 - 180 degrees, with around 120 degrees being the norm.<sep><cls>",
#     "The personification of shinrabansho<sep><cls>",
#     "Naiku: September 25, 1993/September 2013;<sep> Geku: September 27, 1993/September 2013<sep><cls>",
#     "In 1957, he moved to Tokyo Eiga Haikyu<sep> (Tokyo film distribution company) affiliated with Toho Co., Ltd.<sep><cls>",
#     "He was the fourth son to Moroyashi YAMANA.<sep><cls>", # <-
#     "In the Katsuragi region, construction of famous tumulus started from the middle of the early Kofun period (Tumulus period).<sep><cls>",
#     "Most of the old mechanical dolls seen outside Japan do a simple reciprocation, such as sawing and ax-swinging.<sep><cls>",
#     "He took Hyobu MIKAMI, Sigekuni JUGE and others as his disciples.<sep> On the wall panel, Shachihoko(mythical creature with a tiger's head and the body of a fish)was dynamically painted.<sep><cls>",
#     "On the wall panel, Shachihoko(mythical creature with a tiger's head and the body of a fish)was dynamically painted.<sep><cls>",
#     "Habutae is a traditional Japanese woven cloth, which is said to have originated in the early-modern times.<sep><cls>",
# ]


In [9]:
sentences = [
    "The 1977 South African Grand Prix was a Formula One motor race held at Kyalami on 5 March 1977, won by Niki Lauda of Austria. <sep> The race is principally remembered for the accident that resulted in the deaths of race marshal Frederick Jansen van Vuuren and driver Tom Pryce. <sep> It was also the last race for Carlos Pace, who was killed in an aircraft accident less than two weeks later. <sep> <cls>",
    "James Hunt took his third consecutive pole position, with Carlos Pace alongside and Niki Lauda next. <sep> Hunt led off at the start, with Lauda and local driver Jody Scheckter following him after Pace struggled. <sep> The order stayed put until the seventh lap when Lauda took the lead and was never passed again, with Scheckter taking second from Hunt 11 laps later. <sep> <cls>",
    "On lap 22, the Shadow-Ford of Italian driver Renzo Zorzi retired from the race with engine failure and moved off the track on the left side. <sep> Moments after the car came to a halt on a blind brow, the engine caught fire. <sep> Reacting to the emergency, two fire marshals carrying fire extinguishers ran across the track to put out the blaze as the race continued. <sep> As they did so, Pryce, in the other Shadow car, and Hans-Joachim Stuck, driving a March-Ford, crested the rise. <sep> Stuck's leading car swerved, narrowly missing the first marshal, but Pryce, who was unsighted behind the German, had no time to react before hitting the second marshal, Frederick Jansen van Vuuren, who was killed instantly. <sep> <cls>",
    "At the moment of impact of the car with the marshal, the fire extinguisher that Jansen van Vuuren had been holding hit Pryce's head, killing him instantly and nearly decapitating him. <sep> The Shadow then continued down the main straight at speed with Pryce's body still seated behind the wheel. <sep> The car finally left the track at the first corner, taking the Ligier of Jacques Laffite out of the race in the process. <sep> The incident was filmed by a broadcast crew covering the race. <sep> <cls>",
    "Lauda's Ferrari was barely able to finish the race after his car had picked up part of the wreckage from Pryce's accident in the underside of its monocoque. <sep> This damaged the car's water system and at the end of the race, the team found that only a third of the usual twelve liters of water remained in the system. <sep> Both the warnings for oil pressure and water temperature had been flashing at Lauda for the final 25 laps, in a car which he later described as 'completely gone'. <sep> <cls>",
    "Despite this, Lauda held on to win his first victory since his near-fatal crash at the 1976 German Grand Prix the previous year. <sep> South African Scheckter was second, and Patrick Depailler's six-wheeler took third from Hunt in the closing laps. <sep> At first Lauda announced it was the greatest victory of his career, but when told on the victory podium of Pryce's death, he said that 'there was no joy after that'. <sep> <cls>",
    "Jansen van Vuuren's injuries were so severe that, initially, his body was only identified after the race director had summoned all of the race marshals and Van Vuuren was not among them. <sep> <cls>",
    "The sport reacted with sorrow at the loss of two young men. <sep> Tyrrell mechanic Trevor Foster viewed the incident from a distance, later recalling  <sep> <cls>",
    "David Tremayne, a veteran biographer and motor sports journalist, recalled the feelings of disbelief and horror following the aftermath of the incident;  <sep> <cls>",
    "The event was included in the motor racing film 'The Quick and the Dead'. <sep> <cls>",
    "1977 Spanish Grand Prix <sep> <cls>",
    "The 1977 Spanish Grand Prix was a Formula One motor race held on 8 May 1977 at the Circuito del Jarama near Madrid, Spain. <sep> It was the fifth race of the 1977 World Championship of F1 Drivers and the 1977 International Cup for F1 Constructors. <sep> <cls>",
    "The 75-lap race was won from pole position by American driver Mario Andretti, driving a Lotus-Ford. <sep> Argentinian Carlos Reutemann finished second in a Ferrari, with South African Jody Scheckter third in a Wolf-Ford. <sep> <cls>",
    "Mario Andretti continued his recent form with pole position and Jacques Laffite's Ligier springing a surprise, beating Niki Lauda to second, however Lauda was unable to take part in the race due to a broken rib. <sep> At the start, Andretti took the lead with Laffite following. <sep> Laffite gave chase to Andretti in the early stages but had to pit with a deflating tyre, leaving Andretti to romp to victory with Carlos Reutemann second. <sep> Hunt inherited third but retired with a misfire, giving it to John Watson until he lost places due to a spin, leaving Jody Scheckter to take the final spot on the podium. <sep> <cls>",
    "The race marked the debut of the Williams Grand Prix Engineering team, entering a single March 761 chassis for Belgian driver Patrick Nève, who finished 12th. <sep> <cls>",
    "1977 Monaco Grand Prix <sep> <cls>",
    "The 1977 Monaco Grand Prix was a Formula One motor race held at Monaco on 22 May 1977. <sep> It was the sixth race of the 1977 World Championship of F1 Drivers and the 1977 International Cup for F1 Constructors. <sep> <cls>",
    "The 76-lap race was won by South African driver Jody Scheckter, driving a Wolf-Ford. <sep> It was Scheckter's second victory of the season, and the 100th World Championship race victory for the Ford-backed Cosworth DFV engine. <sep> Austrian Niki Lauda finished second in a Ferrari, with Argentinian teammate Carlos Reutemann third. <sep> <cls>",
    "John Watson took his first career pole for Brabham with Jody Scheckter also on the front row and Carlos Reutemann third. <sep> It was Scheckter who got the best start to beat Watson to the first corner, with Reutemann running third in the early stages until he was passed by his teammate Niki Lauda. <sep> Watson ran second to Scheckter until mid-race when he had to retire with gearbox trouble, allowing Lauda to close in on Scheckter but the latter was flawless and held on to take his second win of the season. <sep> Lauda had to be satisfied with second, with Reutemann completing the podium. <sep> <cls>",
    "Shadow driver Riccardo Patrese made his first career start in the race, qualifying 15th and finishing 9th. <sep> Patrese would drive in a record 257 Grands Prix (256 starts) during his career which ended after the season, a career which included winning at Monaco for Brabham in . <sep> <cls>",
    "1977 Belgian Grand Prix <sep> <cls>",
    "The 1977 Belgian Grand Prix was a Formula One motor race held at Zolder on 5 June 1977. <sep> It was the seventh race of the 1977 World Championship of F1 Drivers and the 1977 International Cup for F1 Constructors. <sep> <cls>",
    "The 70-lap race was won by Swedish driver Gunnar Nilsson, driving a Lotus-Ford. <sep> Austrian driver Niki Lauda finished second in a Ferrari, while Nilsson's fellow Swede, Ronnie Peterson, finished third in a Tyrrell-Ford. <sep> This was to be Nilsson's only Formula One victory, before his career was cut short by cancer and he died in October 1978. <sep> <cls>",
    "In qualifying Mario Andretti took a comfortable pole position with John Watson just beating the second Lotus of Gunnar Nilsson to second. <sep> The race was wet due to rain earlier in the day, and Watson took the lead at the first corner, but later in the first lap, Andretti ran into the back to him while attempting to retake the lead, resulting in both of them spinning out and Jody Scheckter taking the lead. <sep> <cls>",
    "Scheckter led the early stages ahead of Nilsson, whereas Carlos Reutemann made an early charge but spun off. <sep> The track began to dry and everyone had to pit for tyres, and those who stopped early were the ones who benefited, as now Niki Lauda was leading from Jochen Mass and Scheckter was down to fifth. <sep> Mass spun off from second, handing it to Nilsson who then went on to catch and pass Lauda before driving away to his first career victory. <sep> Behind second-placed Lauda, Ronnie Peterson completed the podium for Team Tyrrell after Scheckter retired with an engine failure. <sep> This remains the only F1 race in which two Swedes shared the podium. <sep> <cls>",
    "1977 Swedish Grand Prix <sep> <cls>",
    "The 1977 Swedish Grand Prix was a Formula One motor race held at the Scandinavian Raceway on 19 June 1977. <sep> It was the eighth race of the 1977 World Championship of F1 Drivers and the 1977 International Cup for F1 Constructors. <sep> <cls>",
    "The 72-lap race was won by Frenchman Jacques Laffite, driving a Ligier-Matra. <sep> This was the first Formula One victory for a French team and a French engine, as well as the first all-French victory in the Formula One World Championship. <sep> <cls>",
    "German driver Jochen Mass finished second in a McLaren-Ford, with Argentinian Carlos Reutemann third in a Ferrari. <sep> <cls>",
    "The Swedish race was full of anticipation after Gunnar Nilsson's win last time out, but once again in qualifying, it was his teammate Mario Andretti leading the way from John Watson, with James Hunt heading the second row. <sep> At the start, again it was Watson who led into the first corner followed by Jody Scheckter, but soon Andretti passed both of them and opened up a lead. <sep> Watson and Scheckter battled for second until they collided, with Scheckter having to retire and Watson pitting for repairs. <sep> Hunt took second but he began to drop back and was passed by a charging Jacques Laffite, and then by his teammate Jochen Mass. <sep> Andretti however, was dominant until he had to pit due to a fuel metering problem with two laps left, handing the lead to Laffite who went on to take his first ever win ahead of Mass and Reutemann. <sep> <cls>",
    "Laffite's victory in his Gitanes-sponsored Ligier-Matra marked the first all-French victory in World Championship history. <sep> <cls>",
    "The 1977 French Grand Prix was a Formula One motor race held at Dijon on 3 July 1977. <sep> <cls>",
    ]

# Pre-processing

In [10]:
def encode_ids(sp_model, s):
    assert 'Clancy' not in s and 'Abdallah' not in s
    s = s.replace('<cls>', ' Clancy')
    s = s.replace('<sep>', ' Abdallah')
    ids = sp_model.encode_as_ids(s)
    for i, id_ in enumerate(ids):
        if id_ == sp_model.encode_as_ids(' Clancy')[0]:
            ids[i] = sp_model.piece_to_id('<cls>')
        elif id_ == sp_model.encode_as_ids(' Abdallah')[0]:
            ids[i] = sp_model.piece_to_id('<sep>')
    return ids

In [11]:
preprocessed_sentences = []
ids_list = []
for sentence in sentences:
    s = preprocess_text(sentence, lower=False)
    preprocessed_sentences.append(s)
    ids_list.append(encode_ids(sp_model, s))

# Padding
for ids in ids_list:
    pads = [sp_model.pad_id()] * (seq_length - len(ids))
    ids.extend(pads)
ids = np.array(ids_list, dtype=np.int32)

sample_input_ids = ids.T
sample_input_mask = sample_input_ids == sp_model.pad_id()
sample_segment_ids = np.zeros_like(sample_input_ids)

# Run Model

In [12]:
feed_dict={
    input_ids: sample_input_ids,
    segment_ids: sample_segment_ids,
    input_mask: sample_input_mask
}
sample_seq_out = sess.run(seq_out, feed_dict=feed_dict)

# Tensorboard setting

In [13]:
# Create Meta Data
ids_list = sample_input_ids.T
sentence_ids_list = []
tokens_list = []
for sentence_idx, ids in enumerate(ids_list):
    sentence_ids = [sentence_idx] * len(ids)
    sentence_ids_list.append(sentence_ids)
    
    tokens = []
    for i in ids:
        tokens.append(sp_model.id_to_piece(int(i)))
    tokens_list.append(tokens)

metadata = []
for tokens, sentence_ids in zip(tokens_list, sentence_ids_list):
    for t, sid in zip(tokens, sentence_ids):
        metadata.append((t, sid))
        
embs = sample_seq_out.transpose((1, 0, 2))
embs = embs.reshape(-1, 1024)

In [15]:
metadata[:100]

[('▁The', 0),
 ('▁1977', 0),
 ('▁South', 0),
 ('▁African', 0),
 ('▁Grand', 0),
 ('▁Prix', 0),
 ('▁was', 0),
 ('▁a', 0),
 ('▁Formula', 0),
 ('▁One', 0),
 ('▁motor', 0),
 ('▁race', 0),
 ('▁held', 0),
 ('▁at', 0),
 ('▁Ky', 0),
 ('al', 0),
 ('ami', 0),
 ('▁on', 0),
 ('▁5', 0),
 ('▁March', 0),
 ('▁1977,', 0),
 ('▁won', 0),
 ('▁by', 0),
 ('▁Nik', 0),
 ('i', 0),
 ('▁La', 0),
 ('uda', 0),
 ('▁of', 0),
 ('▁Austria', 0),
 ('.', 0),
 ('<sep>', 0),
 ('▁The', 0),
 ('▁race', 0),
 ('▁is', 0),
 ('▁principally', 0),
 ('▁remembered', 0),
 ('▁for', 0),
 ('▁the', 0),
 ('▁accident', 0),
 ('▁that', 0),
 ('▁resulted', 0),
 ('▁in', 0),
 ('▁the', 0),
 ('▁deaths', 0),
 ('▁of', 0),
 ('▁race', 0),
 ('▁marshal', 0),
 ('▁Frederick', 0),
 ('▁Jan', 0),
 ('sen', 0),
 ('▁van', 0),
 ('▁Vu', 0),
 ('ur', 0),
 ('en', 0),
 ('▁and', 0),
 ('▁driver', 0),
 ('▁Tom', 0),
 ('▁P', 0),
 ('ry', 0),
 ('ce', 0),
 ('.', 0),
 ('<sep>', 0),
 ('▁It', 0),
 ('▁was', 0),
 ('▁also', 0),
 ('▁the', 0),
 ('▁last', 0),
 ('▁race', 0),
 ('▁for', 0)

# Add outputs and metadata to Tensorboard

In [16]:
writer = SummaryWriter(log_dir)

In [17]:
writer.add_embedding(embs, metadata, metadata_header=('token', 'sentence_id'))

# Appenedix: Sentencepiece problem
* Is there any way to change --control_symbols to --user_defined_symbols?
* https://github.com/google/sentencepiece/issues/215

In [19]:
sp_model.encode_as_pieces('<sep>')

['▁<', 's', 'ep', '>']